In [120]:
import bql
import pandas as pd
import numpy as np
from itertools import islice
from datetime import datetime, timedelta
from bql.util import get_time_series
from collections import OrderedDict
import matplotlib.pyplot as plt

# Instantiate an object to interface with the BQL service
bq = bql.Service()

In [121]:
import bqplot as bqp
import bqviz as bqv
from bqplot.interacts import BrushSelector
from bqwidgets import TickerAutoComplete
from bqviz import BarPlot, HistPlot
from collections import OrderedDict
from ipywidgets import *
from ipydatagrid import DataGrid, TextRenderer
from IPython.display import display
from bqplot import Axis, LinearScale, OrdinalScale, Scatter, Figure, Tooltip
from functools import partial



In [122]:
#NECESSARY FUNCTIONS

#1: BQL rank function
def rank_func(factor):
      return factor.group().znav().rank().ungroup().applyPreferences(Currencycheck="ignore")

In [123]:
#2: Factor calculations using BQL
def factor_calc(term, index, currency, roe1, roe2):
    # Selected universe
    bq_univ = bq.univ.members(selected_index)

    # Price to book
    bq_px_to_book = bq.data.px_last(fill='PREV',currency=selected_currency) / bq.data.book_val_per_sh(fa_period_type=selected_term,currency=selected_currency)

    # Define a composite BQL data item with each BQL Item component of the 2-year average ROE
    bq_avg_roe = (bq.data.return_com_eqy(fa_period_offset=roe_term_1) 
                 + bq.data.return_com_eqy(fa_period_offset=roe_term_2)) / 2

    # Operating cash flow and total assets
    cash_from_oper = bq.data.cf_cash_from_oper(fa_period_type=selected_term)
    tot_asset = bq.data.bs_tot_asset(fa_period_type=selected_term)

    # Operating CF / Total Assets 
    bq_cash_per_asset = cash_from_oper / tot_asset
    
    #count index members and define thresholds for future ranking
    id_count = bq.data.id().group().count()
    bq_res = bq.execute(bql.Request(bq_univ, {'COUNT': id_count}))
    num_of_members = bq_res[0].df()['COUNT']

    threshold_percentage = 0.90
    threshold_rank = int(threshold_percentage * num_of_members)
    
    #Ranking factors 
    factor_ranks = [rank_func(bq_avg_roe),
                rank_func(bq_px_to_book),
                rank_func(bq_cash_per_asset)]

    #Selecting criteria
    criteria = [factor_rank <= threshold_rank for factor_rank in factor_ranks]

    # Combine the three criteria items with AND clause
    criteria_final = criteria[0]    
    for i in range(1, len(criteria)): criteria_final = bq.func.and_(criteria_final, criteria[i])        

    # Filter the universe by criteria
    filtered_univ = bq.univ.filter(bq_univ, criteria_final)
    
    # Define an ordered dictionary
    items_ordered_dict = OrderedDict()

    # Populate the dictionary with our screening factors and custom header labels 
    items_ordered_dict['AVE_ROE'] = bq_avg_roe
    items_ordered_dict['PX_BOOK_VALUE'] = bq_px_to_book
    items_ordered_dict['CASH_PER_ASSET'] = bq_cash_per_asset
    
    # Generate and execture the requests 
    bq_request = bql.Request(filtered_univ, items_ordered_dict, with_params={"mode": "cached"})

    bq_res = bq.execute(bq_request)

    # Dataframe populated with results
    bq_result_df = pd.concat([sir.df()[sir.name] for sir in bq_res], axis=1)
    
    return bq_result_df

In [124]:
#3: Delta MAV Momentum Function

#index name as string
def d_mav(number_of_days,index_name):
    
    start_date = datetime.today()-timedelta(days=number_of_days)
    end_date = datetime.today()
    
    date_range = bq.func.range(start_date,end_date)
    px_last = bq.data.px_last(dates=date_range)
    univ = bq.univ.members(index_name)

    request = bql.Request(univ, px_last)
    response = bq.execute(request)

    df = response[0].df()
    df = df.drop("CURRENCY", axis=1)
    df = df.rename(columns={df.columns[1]:'Last'})
    df = df.sort_values('ID')
    df = df.dropna()
    df = df.reset_index().set_index('DATE')
    mav = df.sort_values(['ID','DATE']).groupby('ID').rolling("3D")['Last'].mean().to_frame()
    mav = mav.rename(columns={mav.columns[0]:'Average'})
    mav['d_Diff'] = mav.groupby(['ID'])['Average'].diff()
    mav = mav.groupby('ID').d_Diff.apply(lambda x: pd.Series([(x < 0).sum(), (x >= 0).sum()])).unstack()
    mav = mav.rename(columns={0:'Negative', 1:'Positive'})
    mav['Direction'] = mav['Positive']-mav['Negative']
    mav = mav.sort_values('Direction', ascending=False)
    #mav_top = mav.nlargest(30,'Direction')
    #mav_bottom = mav.nsmallest(30, 'Direction')
    return mav


In [125]:
def pe_calc(selected_universe, selected_term, minPE):
    #variables will be dataframe, string, integer
    # Create a new universe based on passed on dataframe
    bq_univ_selection = bq.univ.list(selected_universe.index)

    # Define a custom column name
    col_name = 'PE_RATIO'

    # BQL data item for LTM price-to-earnings ratio
    bq_pe_ratio = bq.data.pe_ratio(fa_period_type=selected_term)

    # Price to Earnings Ratio is greater than 0
    criteria = bq_pe_ratio >= minPE

    # Filtered universe based on selection and criteria
    filtered_univ = bq.univ.filter(bq_univ_selection, criteria) 

    # Generate and executed request 
    bq_request = bql.Request(filtered_univ, {col_name:bq_pe_ratio}, with_params={"mode": "cached"})
    bq_res = bq.execute(bq_request)

    # Define a new DataFrame using response get function
    bq_result_df = bq_res.get(col_name).df()

    # Select 100 securities with the lowest ratios
    bq_result_df = bq_result_df[col_name].sort_values(ascending=True)
    bq_result_df = bq_result_df.head(100)

    # Print the DataFrame to see the result
    bq_result_df.to_frame()
    return bq_result_df

In [126]:
#calculates simple moving average for a given timeframe
def sMav_Distance(number_of_days, index_name):
    
    price = bq.data.px_last(dates=bq.func.range('-'+str(number_of_days)+'d','0d'))
    avg_price = price.avg()
    rolling_avg = avg_price.rolling(iterationdates=bq.func.range('-'+str(number_of_days)+'d','0d', frq='d'))
    smav_distance = price/rolling_avg

    req = bql.Request(index_name, {'Avg Price to spot':smav_distance})
    res = bq.execute(req)
    df = res[0].df()
    df = df.drop("CURRENCY", axis=1)
    df = df.dropna()

    return df

In [127]:
# Define a class to control the visualization and user interface
class FactorVisualization:
    def __init__(self):
        # Initialize the chart with a default ticker
        self.data_chart = pd.DataFrame()
        self.chart_dropdown_x = Dropdown(description='X-Axis',layout=Layout(width='380px'))
        self.chart_dropdown_y = Dropdown(description='Y-Axis',layout=Layout(width='380px'))
        
        self.x_sc = LinearScale()
        self.y_sc = LinearScale()  
        self.tt = Tooltip(fields=['name','x', 'y'], formats=['', '.2f','.2f']) 
        self.scatter = Scatter(scales= {'x': self.x_sc, 'y': self.y_sc}, colors=['dodgerblue'],
                        tooltip=self.tt, unhovered_style={'opacity': 0.5})
        self.scatter.height = '%200'

        self.ax_x = Axis(scale=self.x_sc)
        self.ax_y = Axis(scale=self.y_sc, orientation='vertical', tick_format='0.2f')        
        self.fig = Figure(marks=[], axes=[self.ax_x, self.ax_y],
                          animation_duration=1000,
                          padding_x=0,layout = {'width':"100%",'height':"500px"})
        
        self.data_grid = DataGrid(dataframe=pd.DataFrame(), 
                                  layout={'width':'800px','height':'210px', 'margin': 'auto'}, 
                                  base_column_size=100,
                                  column_widths={'ID': 100},
                                  default_renderer=TextRenderer(horizontal_alignment='center'))
        
        self.box = VBox([HBox([self.fig]),
                         HBox([self.chart_dropdown_x,self.chart_dropdown_y])
                        ])
        
        display(self.box)
    
    # Define a method to update scatter chart
    def plot_scatter(self, x_data, y_data, label_data,x_name,y_name):
        distance_x = max(x_data) - min (x_data)
        distance_y = max(y_data) - min (y_data)
        self.x_sc.min = min (x_data) - (distance_x)/10
        self.x_sc.max = max(x_data) + (distance_x)/10
        self.y_sc.min = min (y_data) - (distance_y)/10
        self.y_sc.max = max(y_data) + (distance_y)/10
        self.x_data = x_data
        self.y_data = y_data
        self.label_data = label_data
        self.tt.labels=['Company',x_name,y_name]
        self.scatter.x = x_data
        self.scatter.y = y_data
        self.scatter.names = label_data
        self.fig.title = "Scatter Chart: " + x_name + "  v " + y_name
        self.ax_x.label = x_name
        self.ax_y.label = y_name
        self.fig.marks = [self.scatter]
    
    # Define a method to control default chart state
    def populate_default_chart(self,df):
        self.data_chart = df
        options_list = list(df.columns)
        options_list.append(' ')
        self.chart_dropdown_x.options = options_list
        self.chart_dropdown_y.options = options_list
        self.chart_dropdown_x.value =  options_list[0] 
        self.chart_dropdown_y.value =  options_list[1]
        x_data = df[df.columns[0]].values
        y_data = df[df.columns[1]].values
        label_data = df[df.columns[0]].index
        self.plot_scatter(x_data,y_data,label_data,list(df.columns)[0],list(df.columns)[1])
        self.chart_dropdown_x.observe(self.on_change)
        self.chart_dropdown_y.observe(self.on_change)
    
    # Define a handler for axis changes on chart
    def on_change(self,change):
        if change['type'] == 'change' and change['name'] == 'value':
            if self.chart_dropdown_x.value == ' ' or self.chart_dropdown_y.value == ' ':
                pass
            else:
                x_data = self.data_chart[self.chart_dropdown_x.value].values
                y_data = self.data_chart[self.chart_dropdown_y.value].values
                label_data = self.data_chart[self.data_chart.columns[0]].index
                self.plot_scatter(x_data,y_data,label_data,self.chart_dropdown_x.value,self.chart_dropdown_y.value)

# Define a handler for when the 'Generate Scores' button is clicked
def handle_click(sender, vs): 
    vs.chart_dropdown_x.value =  ' '
    vs.chart_dropdown_y.value =  ' '
    factor_score = get_data(dropdown_universe.value,dropdown_model.value,params)
    vs.populate_default_chart(factor_score)
    factor_score_for_grid = factor_score
    factor_score_for_grid.columns = [x.replace('.','_') for x in list(factor_score_for_grid.columns.values)]
    vs.data_grid.data = factor_score_for_grid.round(2)
    display(vs.data_grid)

In [128]:
# Define universe mappings to be used in dropdown menus
universe = {
    "XU100": bq.univ.members('XU100 Index'),
    "Test": bq.univ.members('INDU Index'),         
}

univ_list = list(universe.keys())

In [129]:
#LTM: 12 months. #STM: 6 months. Some factors won't run on STM
selected_term = 'LTM' 
#Index ticker in Bloomberg
selected_index = 'XU100 Index' 
#Currency ticker. Some EMs not supported incl TRY
selected_currency = 'USD'
#roe terms are years looking back. Integer only. Type is string. term 1 < term 2
roe_term_1 = '-1' 
roe_term_2 = '-2' 

#days of momentum measurement
st_days = 21
mt_days = 182

#Populate the momentum dataframe
mt_momentum = d_mav(mt_days,selected_index)
st_momentum =d_mav(st_days,selected_index)
df_momentum = pd.concat([mt_momentum, st_momentum])

# Calculate the spot index distance to simple moving average price
st_smav_dist = sMav_Distance(st_days, selected_index)
mt_smav_dist = sMav_Distance(mt_days, selected_index)
df_smav_dist = pd.concat([mt_smav_dist, st_smav_dist])

#populate the factors dataframe
factors = factor_calc(selected_term, selected_index, selected_currency, roe_term_1, roe_term_2)

#populate the P/E dataframe
min_pe_stocks = pd.DataFrame(pe_calc(factors, selected_term, 0))

#rank factors, momentum, P/E dataframes
ranked_momentum = df_momentum.groupby(df_momentum.index).mean().rank(pct=True, method='average', na_option='bottom')
ranked_momentum = ranked_momentum.sort_values(['Direction'], ascending=False)

ranked_factors = factors.groupby(factors.index).mean().rank(pct=True, method='average', na_option='bottom')
ranked_factors['Weighted Rank'] = (ranked_factors * [0.20,0.40,0.40]).sum(axis=1)
ranked_factors = ranked_factors.sort_values(['Weighted Rank'], ascending=False)

ranked_pe = min_pe_stocks.groupby(min_pe_stocks.index).mean().rank(pct=True, method='average', na_option='bottom')
ranked_pe = ranked_pe.sort_values(['PE_RATIO'], ascending=True)
ranked_pe['Rank'] = 1 - ranked_pe['PE_RATIO']

In [130]:
#check if the index is at premium or discount for the last 5 days
#check premium/discount
index_price = df_smav_dist['Avg Price to spot'].tail(5).mean()

#choose multi-factor weights based on premium/discount level
if index_price >=1.01:
    (w_mom, w_fac, w_pe) = (0.6, 0.3, 0.1)
elif 0.99 <= index_price < 1.01:
    (w_mom, w_fac, w_pe) = (0.4, 0.4, 0.2)
else: 
    (w_mom, w_fac, w_pe) = (0.2, 0.4, 0.4)


final_momentum = (ranked_momentum['Direction'] * w_mom)
final_factor = (ranked_factors['Weighted Rank'] * w_fac) 
final_pe = (ranked_pe['Rank'] * w_pe)


In [131]:
#create results dataframe
result_df = pd.concat([final_momentum, final_factor, final_pe])
frame = {'Momentum': final_momentum, 'Factor': final_factor, 'P/E': final_pe}
result_df = pd.DataFrame(frame).dropna()

In [132]:
result_df['Final Score'] = result_df.sum(axis=1)

In [133]:
result_df.head()

,Momentum,Factor,P/E,Final Score
AEFES TI Equity,0.249,0.074074,0.038667,0.361741
AKBNK TI Equity,0.483,0.037778,0.093333,0.614111
AKCNS TI Equity,0.075,0.104444,0.021333,0.200778
AKSA TI Equity,0.513,0.242222,0.061333,0.816556
AKSEN TI Equity,0.432,0.074815,0.046667,0.553481


In [134]:
#Pick the top 30 stocks from results
top30_list = {'Final Factor Score' : result_df.head(30).sum(axis=1)}
top30_stocks = pd.DataFrame(top30_list).sort_values('Final Factor Score', ascending=False)
t_date = datetime.now().strftime('%Y-%m-%d')
top30_stocks.to_excel('top30_stocks_' + t_date + '.xlsx')


In [141]:
top30_stocks.head(10)

,Final Factor Score
AKSA TI Equity,1.633111
CANTE TI Equity,1.589630
EKGYO TI Equity,1.526370
ARDYZ TI Equity,1.398148
ENKAI TI Equity,1.341852
FROTO TI Equity,1.341704
DOHOL TI Equity,1.261630
AKBNK TI Equity,1.228222
CIMSA TI Equity,1.227333
ALARK TI Equity,1.209778


In [136]:
# Define and display the input widgets
dropdown_universe = Dropdown(options=univ_list,description='Analysis Universe',style={'description_width':'initial'},value=univ_list[1])
#dropdown_model = Dropdown(options=model_list,description='Factor Model',value=model_list[0])
button_update = Button(description="Generate Scores", button_style='success')
interface = VBox([HBox([dropdown_universe, button_update])]) 
display(interface)

# Initialize visualization class and populate the chart
vs =  FactorVisualization()
factor_score = result_df
vs.populate_default_chart(factor_score)
button_update.on_click(partial(handle_click,vs=vs))

# Set up and populate the datagrid layout
factor_score_for_grid = factor_score
factor_score_for_grid.columns = [x.replace('.','_') for x in list(factor_score_for_grid.columns.values)]
vs.data_grid.data = factor_score_for_grid.round(2)
display(vs.data_grid)


DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, base_column_size=100, column_widths=…